In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df = df[0:50000]

In [4]:
df.drop(["link"], axis = 1, inplace = True)
df.rename(columns = {"text":"lyrics"}, inplace = True)
df.drop_duplicates(subset="song", inplace=True)
df.reset_index(drop = True, inplace = True)

In [5]:
df

,artist,song,lyrics
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...
...,...,...,...
39461,Ramones,Don't Bust My Chops,I'm sick and tired of you calling me names \r...
39462,Ramones,Don't Come Close,You know it's generally known \r\nYou got eve...
39463,Ramones,Don't Pour Out All The Oil,Verse: \r\nDon't pour out all the oil \r\nIt...
39464,Ramones,Eat That Rat,One-two-three-four \r\n \r\nYou don't want t...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39466 entries, 0 to 39465
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  39466 non-null  object
 1   song    39466 non-null  object
 2   lyrics  39466 non-null  object
dtypes: object(3)
memory usage: 925.1+ KB


In [7]:
df["combined_features"] = df["artist"]+ " "+df["song"]+" "+df["lyrics"]

In [8]:
df

,artist,song,lyrics,combined_features
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...","ABBA Ahe's My Kind Of Girl Look at her face, i..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...","ABBA Andante, Andante Take it easy with me, pl..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...,ABBA As Good As New I'll never know why I had ...
3,ABBA,Bang,Making somebody happy is a question of give an...,ABBA Bang Making somebody happy is a question ...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,ABBA Bang-A-Boomerang Making somebody happy is...
...,...,...,...,...
39461,Ramones,Don't Bust My Chops,I'm sick and tired of you calling me names \r...,Ramones Don't Bust My Chops I'm sick and tired...
39462,Ramones,Don't Come Close,You know it's generally known \r\nYou got eve...,Ramones Don't Come Close You know it's general...
39463,Ramones,Don't Pour Out All The Oil,Verse: \r\nDon't pour out all the oil \r\nIt...,Ramones Don't Pour Out All The Oil Verse: \r\...
39464,Ramones,Eat That Rat,One-two-three-four \r\n \r\nYou don't want t...,Ramones Eat That Rat One-two-three-four \r\n ...


In [9]:
df.combined_features[0]

"ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [10]:
for i in range(0,5):
    print("*********************************************\n")
    print(df.combined_features[i])

*********************************************

ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  
  
And when we go for a walk in the park  
And she holds me and squeezes my hand  
We'll go on walking for hours and talking  
About all the things that we plan  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?


*********************************************

ABBA Andante, Andante Take it easy with me, please  
Touch me gently like a summer evening breeze  
Take your time, m

In [11]:
import re
import nltk
from nltk.corpus import stopwords
def cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

In [12]:
df["combined_features"] = df["combined_features"].apply(func = cleaning)

In [13]:
for i in range(0,5):
    print("\n*********************************************\n")
    print(df.combined_features[i])


*********************************************

abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could

*********************************************

abba andante andante take easy please touch gently like summer evening breeze take time make slow andante andante let feeling grow make fingers soft light let body velvet night touch soul know andante andante go slowly music music song song music song play time time make strong play cause making strong make sing make sound make sing make andante andante tread lightly ground andante andante oh please let shimmer eyes like feeling thousand butterflies please talk go play andante andante let float away music music song 

In [14]:
df.combined_features[0]

'abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could'

In [15]:
# cv = CountVectorizer()
# count_matrix = cv.fit_transform(df["combined_features"])
# print(count_matrix.toarray())

In [16]:
# count_matrix.toarray().shape

In [17]:
tfv = TfidfVectorizer(max_features = 10000)

In [18]:
tfv_matrix = tfv.fit_transform(df["combined_features"])

In [19]:
tfv_matrix.shape

(39466, 10000)

In [20]:
# tfv_matrix.toarray()

In [21]:
cosine_sim = cosine_similarity(tfv_matrix)

In [22]:
song_user_likes = "Hope"
song_index = df[df.song == song_user_likes].index[0]

In [23]:
song_index

659

In [24]:
similar_songs = list(enumerate(cosine_sim[song_index]))

In [25]:
similar_songs[0:5]

[(0, 0.04092533924868175),
 (1, 0.01746788081317406),
 (2, 0.04942975440682899),
 (3, 0.0065672233749841415),
 (4, 0.007298608643914333)]

In [26]:
sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)

In [27]:
sorted_similar_songs[0:5]

[(659, 1.0000000000000002),
 (8068, 0.5784493048716957),
 (37444, 0.5647907063872308),
 (23056, 0.45309671761554454),
 (20793, 0.4242324884462278)]

In [28]:
for song in sorted_similar_songs[1:11]:
        similar_songs = df[df.index == song[0]]["song"].values[0]
        print(similar_songs)

Who Do You Love, I Hope
Here's Hope
I Hope You Find It
Be Alright
I Want To Be With You Tonight
Alright
Hope It's Too Late
If He's Ever Near
Hope And Pray
Want


In [29]:
def recommended_songs(song_user_likes):
    song_index = df[df.song == song_user_likes].index[0]
    similar_songs = list(enumerate(cosine_sim[song_index]))
    sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)
    
    for song in sorted_similar_songs[1:11]:
        similar_songs = df[df.index == song[0]]["song"].values[0]
        print(similar_songs)

In [30]:
recommended_songs("Hope")

Who Do You Love, I Hope
Here's Hope
I Hope You Find It
Be Alright
I Want To Be With You Tonight
Alright
Hope It's Too Late
If He's Ever Near
Hope And Pray
Want
